In [12]:
import numpy as np

$H_n = 2^n n! = (2n)!! $ is number of darts in n-dimensional hypercube, n = 0...

In [13]:
# number of dart in n-dimensional hypercubes
H = [2**i * np.math.factorial (i) for i in range (11)]
H

[1, 2, 8, 48, 384, 3840, 46080, 645120, 10321920, 185794560, 3715891200]

In [14]:
# How many bits are needed to represent dart indices (signed because we want to store deltas in the same table, too)

for n in range (1,11):
    nDarts = 2**n * np.product (range (1,n+1))  # n^2 n! darts of hypercube
    print (f'{n} {nDarts} {np.ceil (1 + np.log2 (nDarts)):.0f}')

1 2 2
2 8 4
3 48 7
4 384 10
5 3840 13
6 46080 17
7 645120 21
8 10321920 25
9 185794560 29
10 -579076096 nan


C:\Users\CARMINE\AppData\Local\Temp/ipykernel_22740/626728141.py:4: RuntimeWarning: overflow encountered in long_scalars
  nDarts = 2**n * np.product (range (1,n+1))  # n^2 n! darts of hypercube
C:\Users\CARMINE\AppData\Local\Temp/ipykernel_22740/626728141.py:5: RuntimeWarning: invalid value encountered in log2
  print (f'{n} {nDarts} {np.ceil (1 + np.log2 (nDarts)):.0f}')


In [15]:
import itertools

def no_i_cells (i, dimensions):
    """Number of i-cells in n-dimensional cubical complex given by n dimensions
    
    Works roughly as follows:
        Sum up products of all possible i-subsets times their n-i compplements increased bu 1. 
        Figure out what happens for an SxRxC 3D-volume 
        
        # 0 cells = (S+1)*(R+1)*(C+1)
        # 1 cells = S*(R+1)*(C+1) + R*(S+1)*(C+1) + C*(S+1)*(R+1)
        # 2 cells = R*C*(S+1) + S*C*(R+1) + S*R*(C+1)
        # 3 cells = S*R*C
        
    Params:
        i - dimension of cells in query
        dimensions - np.array of n dimensions
    """

    n = len (dimensions)
    N = {i for i in range (n)}
    s = 0
    for i0 in (itertools.combinations (np.arange (n), i)):
        i0 = list (i0)
        i1 = list (N - set (i0))
        d0 = dimensions[i0]
        d1 = dimensions[i1] + 1
        p0 = np.product (d0)
        p1 = np.product (d1)

        s += p0*p1
#     print (s)
    return s

# $n$: dimensionslity and shape

In [16]:
n          = np.random.randint (2,6) #  up to 5 dimensions, otherwise it take long
dimensions = np.random.randint (1, 5, size = n) # shape of the voxel scene 

# n, dimensions = 7, np.array ([1,1,1,1,1,1,1], dtype=int)
n, dimensions = 5, np.array ([3,1,3,1,2],     dtype=int)
# n, dimensions = 3, np.array ([2,2,2],     dtype=int)
n, dimensions = 2, np.array ([2,3],     dtype=int)



dim_str = "x".join( [str(i) for i in dimensions])

print (f"We'll construct an unbounded {n}-Gmap of {dim_str}+1 {n}-cells. Membranes have {2*H[n-1]} darts.")
print ('We expect to see:')
for i in range (0,1+n):
    print (f'   # {i}-cells ... {no_i_cells (i,dimensions)}{" interior + 1 background" if i==n else ""}')

We'll construct an unbounded 2-Gmap of 2x3+1 2-cells. Membranes have 4 darts.
We expect to see:
   # 0-cells ... 12
   # 1-cells ... 17
   # 2-cells ... 6 interior + 1 background


In [17]:
# Radix weights to be used in dart to tuple and vice versa

R = np.hstack ((dimensions+1, [H[n], 1]))
W = np.cumproduct( R[::-1]) [::-1]
np.vstack ((R,W))

array([[ 3,  4,  8,  1],
       [96, 32,  8,  1]])

# $D$: Set of darts in all membranes

In [18]:
# iterate over all positions

D = set()

import itertools
for p in itertools.product( *(range (m+1) for m in dimensions)):
    GE = p >= dimensions
#     print (p, GE, end=' ')
    if sum(GE) == 0:
#         print ('adding all membranes : ', end = ' ')
        f,t = 0,H[n]
    if sum(GE) == 1:
        a = np.argmax(GE)
#         print (f'adding   {a}-membrane  : ', end = ' ')
        f,t = a * (H[n] // n), (a+1) * (H[n] // n)
    if sum(GE) >  1:
#         print ('... skipping ', end = '')
        f,t = -1,-1
#     print (f,t,end=' ... ')
    for s in range (f,t):
        tup = p + (s,)
        dart = tup @ W[1:]
#         print (dart, end=' ')
        assert not dart in D
        D |= {dart}
        
#     print()

# Involutions

In [19]:
LUT_ALPHA_STAR        = np.loadtxt(f'LUT/LUT_{n}D_alpha_star.tsv',            dtype=np.int64)
LUT_DD                = np.loadtxt(f'LUT/LUT_{n}D_alpha_{n-1}_scenarios.tsv', dtype=np.int64).reshape(-1,3,n+1)
LUT_PosDartDescriptor = np.loadtxt(f'LUT/LUT_{n}D_PosDartDescriptor.tsv',     dtype=np.uint8)

In [20]:
# alphas for i != n-1

m = dimensions
scenarios = [
    [2,2], # if bounding cell test fails  ( = 0)
    [1,0], # if bounding cell test passes ( = 1), then [corner failed, corner passed]
]
def alpha_i (i,d):
    tup = d % W[:-1] // W [1:]  
    p,s = tup [:n], tup[-1]
    
    if i == n-1:
        x_n1, I_n1, x_n2, I_n2 = LUT_PosDartDescriptor [s,:4]   # first 4 descriptors

#       # i3: original scenario index
#         i3 = 2
#         if p[x_n1] == m [x_n1] * (1 - I_n1):
#             i3 = 1
#             if p[x_n2] == I_n2 * (m[x_n2]-1):
#                 i3 = 0
                
        # i3: branch-less-coded scenario index
        b =  int( p[x_n1] ==  m [x_n1]    * (1 - I_n1) )  # boundary 
        c =  int( p[x_n2] == (m[ x_n2]-1) *      I_n2  )  # corner
        i3 = scenarios [b][c]

        p += LUT_DD [s,i3,:n]
        s  = LUT_DD [s,i3, n]
    else:
        s = LUT_ALPHA_STAR [s,i]
        
    tup [:n],tup[-1] = p,s
    return tup @ W[1:] # tuple x radix

### involution checks

In [21]:
# # involution checks are performed during gmpa construction, too

# if n == 2:
#     assert all ([alpha_0(alpha_0 (d)) == d for d in D])
#     assert all ([alpha_1(alpha_1 (d)) == d for d in D])
#     assert all ([alpha_2(alpha_2 (d)) == d for d in D])

#     assert all ([alpha_0(alpha_2(alpha_0(alpha_2(d)))) == d for d in D])
# if n == 3:
#     assert all ([alpha_0(alpha_0 (d)) == d for d in D])
#     assert all ([alpha_1(alpha_1 (d)) == d for d in D])
#     assert all ([alpha_2(alpha_2 (d)) == d for d in D])
#     assert all ([alpha_3(alpha_3 (d)) == d for d in D])

#     assert all ([alpha_0(alpha_2(alpha_0(alpha_2(d)))) == d for d in D])
#     assert all ([alpha_0(alpha_3(alpha_0(alpha_3(d)))) == d for d in D])
#     assert all ([alpha_1(alpha_3(alpha_1(alpha_3(d)))) == d for d in D])

# Sanity check with array-based nGmaps

In [22]:
from combinatorial.gmaps import nGmap

In [23]:
A = np.full ((1+n,1+max(D)), -1)  # initialize with invalid darts (-1)

for d in D:
    for i in range (1+n):
        A[i,d] = alpha_i(i,d)


In [24]:
g = nGmap.from_alpha_array (A)

assert g.is_valid

# check the number of i-cells for given dimension
assert g.no_ccs       == 1

for i in range (1+n):
    assert g.no_i_cells (i) == no_i_cells (i,dimensions) + (i == n) # count the extra background cell

print (f'The {dim_str} hyper-volume resulted in unbounded', g, f' (note the already counted-in background {n}-cell)')

The 2x3 hyper-volume resulted in unbounded 2-gMap of 68 darts:
  # 0-cells: 12
  # 1-cells: 17
  # 2-cells: 7
  # ccs    : 1
  (note the already counted-in background 2-cell)


# Sequential membrane numbering

Let's consider a $m_0 \times  m_1 \times  ...  \times m_{n-1}$ hypervolume.

- There will be $(1+m_a) \prod_{i \neq  a} m_i$ membranes orthogonal to axis $a$.

- Hypercube will have $2n$ membranes. Each (double) membrane will have
$
2 \frac{H_n}{2n} 
= \frac{H_n}{n}
= \frac{2^n n!}{n}
= 2 \cdot 2^{n-1} (n-1)!
= 2 H_{n-1} 
$

darts. 

- there will be $k_n = 2 H_{n-1} \cdot (1+m_a) \cdot \prod_{i \neq  a} m_i $ darts orthogonal to axis $a$, c.f., the $n$ row-wise products of matrix $R$.

- CS is the cumulative sum of vector $[0, k_0 ... k_{n-1}]$. It is used to:
    - offset axis-restricted dart number to the axis-corresponding "bin" 
    - determine, given global dart $d$, which axis $a$ we are working with.

## Radix per axis



- $R$ is the following $n \times (n+2)$ matrix

$
R = 
\begin{pmatrix}
1+m_0   &   m_1   & \cdots & m_{n-1}    & 2 H_{n-1} & 1  \\
  m_0   & 1+m_1   & \cdots & m_{n-1}    & 2 H_{n-1} & 1  \\
\vdots  & \vdots  & \ddots & \vdots     & 2 H_{n-1} & 1  \\
  m_0   &   m_1   & \cdots & 1+m_{n-1}  & 2 H_{n-1} & 1  \\
\end{pmatrix}
$

- $W$ is row-wise cummulative product of $R$. It is used to compute the radix machinery.



In [25]:
R = np.hstack ((
    np.repeat ([dimensions], n, axis=0) + np.eye (n, dtype=int),
    np.full (n,2*H[n-1]).reshape (n,-1),
    np.full (n,1).reshape (n,-1)
))

W = np.cumproduct( R[:,::-1], axis=1) [:,::-1]
W

# n_membranes = [np.product (dimensions + np.eye (n, dtype=np.uint8) [i]) for i in range (n)]
# assert sum (n_membranes) * 2*H[n-1] == g.n_darts
# CS = np.cumsum ([0] + n_membranes)
# CS *= 2*H[n-1]



darts_per_axis = np.product (R, axis = 1)  # k_i
assert darts_per_axis.sum() == g.n_darts
CS = np.hstack (([0], np.cumsum (darts_per_axis)))


In [26]:
def ps2d (p,s):
    '''(p,s) to dart'''
    a = s // (2*H[n-1]) # which axis
    tup = np.zeros (1+n, dtype=int)
    tup [:n] = p
    tup [-1] = s % (2*H[n-1])
    d  = W[a][1:] @ tup
    d += CS[a]
    return d

In [27]:
def d2ps (d):
    '''dart to (p,s)'''
    a = (d >= CS[1:]).sum() # which axis
    d -= CS[a]
    
    tup = d % W[a][:-1] // W[a][1:]
    p,s = tup [:n], tup [-1]
    s += a * (2*H[n-1])
    return p,s

In [28]:
# test ps2d
# for each axis
# for each position (including +1 along that axis)
for a in range (n):
    for p in itertools.product( *(range (m) for m in dimensions + np.eye (n, dtype=int)[a])):
        for s in range (a*(2*H[n-1]), (a+1)*(2*H[n-1])):
            d = ps2d (p,s)
            pp,ss = d2ps(d)
            assert (pp == p).all()
            assert ss == s


In [29]:
for d in range (g.n_darts):
    p,s = d2ps(d)
    assert (ps2d (p,s) == d)
#     if s == 32:
#         print (f'{d:2} -> [{p[0]},{p[1]},{p[2]}] {s}')

# Sound

to confirm all the cells of the notebook have run w/o error when rerstarting the kernel and re-running the notebook.

In [30]:
from IPython.lib.display import Audio

framerate = 4410
play_time_seconds = 1


t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)

# import os
# # os.system('''say "All cells passed"''');
# os.system('say -v Laura "Experiment úspešný!"');